# The TxTl Toolbox in BioCRNpyler

### A recreation of the original MATLAB TxTl Toolbox, as seen in [Singhal et al. 2020](https://www.biorxiv.org/content/10.1101/2020.08.05.237990v1)

This tutorial shows how to use the EnergyTxTlExtract Mixture with a parameter file derived from the paper above. This Mixture is a simplification of the models used in the original toolbox. Notable changes include:
1. Using only a single nucleotide species NTPs (instead of GTP, ATP, UTP, and CTP)
2. A slightly different NTP regeneration Mechanism which explicitly incorporates the amount of fuel, 3PGA, put into the extract and metabolic leak of the extract.
3. Degredation of RNA bound to ribosomes (which releases the ribosome).
4. A modification of the Energy consumption reactions for Transcription and Translation so that there is only a single binding reaction.

## The CRN displayed below shows the energy utilization process model

In [3]:
import numpy as np
import pylab as plt
from biocrnpyler import *

#A = DNAassembly("A", promoter = "P", rbs = "rbs")
E = EnergyTxTlExtract(parameter_file = "txtl_toolbox_parameters.txt")

CRN = E.compile_crn()

print(CRN.pretty_print())

Species (6) = {0. protein[RNAP] init_conc = 0, 1. protein[Ribo] init_conc = 0, 2. protein[RNAase] init_conc = 0, 3. metabolite[amino_acids] init_conc = 0, 4. metabolite[NTPs] init_conc = 0, 5. metabolite[3PGA] init_conc = 0}

Reactions (2) = [
0. metabolite[3PGA] --> metabolite[NTPs]
 Kf = k metabolite[3PGA]^n / ( K^n + metabolite[3PGA]^n )
  k=0.02
  found_key=(mech=one_step_hill_pathway, partid=NTPs_production, name=vmax).
  search_key=(mech=one_step_hill_pathway, partid=NTPs_production, name=vmax).
  K=10.0
  found_key=(mech=one_step_hill_pathway, partid=NTPs_production, name=K).
  search_key=(mech=one_step_hill_pathway, partid=NTPs_production, name=K).
  n=1

1. metabolite[NTPs] --> 
 Kf = k metabolite[NTPs]^n / ( K^n + metabolite[NTPs]^n )
  k=1.77e-05
  found_key=(mech=one_step_hill_pathway, partid=NTPs_degredation, name=vmax).
  search_key=(mech=one_step_hill_pathway, partid=NTPs_degredation, name=vmax).
  K=10.0
  found_key=(mech=one_step_hill_pathway, partid=NTPs_degredation, 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
